# Example: Hierarchical Control with a Disturbance

This notebook demonstrates the resilience of the hierarchical control system when faced with an external disturbance. It uses the same setup as the `example_hierarchical_control` notebook, but adds a `RainfallAgent`.

This agent simulates a sudden, heavy rainfall event that adds a significant inflow to the reservoir, starting midway through the simulation. This example showcases how the decoupled agents can react to unexpected events to maintain system stability.

## Simulation Logic

The core control logic is the same as the hierarchical example. A high-level agent manages the setpoint for a low-level agent that controls a gate.

The key addition is the **`RainfallAgent`**, which is configured to:
- Start publishing an inflow message at **t = 300s**.
- Continue for a duration of **200s**.
- Add a constant inflow of **150 m^3/s** to the reservoir during this period.

We expect to see the control system react to this sudden inflow by further opening the gate to try and bring the water level back towards its setpoint.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.central_coordination.collaboration.message_bus import MessageBus
from swp.simulation_identification.disturbances.rainfall_agent import RainfallAgent
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.central_coordination.dispatch.central_dispatcher import CentralDispatcher

# Helper function from the original examples, included for self-containment
def setup_hierarchical_control_system(message_bus, simulation_dt):
    RESERVOIR_STATE_TOPIC = "state.reservoir.level"
    GATE_STATE_TOPIC = "state.gate.gate_1"
    GATE_ACTION_TOPIC = "action.gate.opening"
    GATE_COMMAND_TOPIC = "command.gate1.setpoint"

    gate_params = {'max_rate_of_change': 0.5, 'discharge_coefficient': 0.6, 'width': 10, 'max_opening': 5.0}
    reservoir = Reservoir(name="reservoir_1", initial_state={'volume': 28.5e6, 'water_level': 19.0}, parameters={'surface_area': 1.5e6})
    gate = Gate(name="gate_1", initial_state={'opening': 0.1}, parameters=gate_params, message_bus=message_bus, action_topic=GATE_ACTION_TOPIC)

    reservoir_twin = DigitalTwinAgent(agent_id="twin_reservoir_1", simulated_object=reservoir, message_bus=message_bus, state_topic=RESERVOIR_STATE_TOPIC)
    gate_twin = DigitalTwinAgent(agent_id="twin_gate_1", simulated_object=gate, message_bus=message_bus, state_topic=GATE_STATE_TOPIC)

    pid = PIDController(Kp=-0.8, Ki=-0.1, Kd=-0.2, setpoint=15.0, min_output=0.0, max_output=gate_params['max_opening'])
    lca = LocalControlAgent(agent_id="lca_gate_1", controller=pid, message_bus=message_bus, observation_topic=RESERVOIR_STATE_TOPIC, observation_key='water_level', action_topic=GATE_ACTION_TOPIC, dt=simulation_dt, command_topic=GATE_COMMAND_TOPIC, feedback_topic=GATE_STATE_TOPIC)

    rules = {'flood_threshold': 18.0, 'normal_setpoint': 15.0, 'flood_setpoint': 12.0}
    dispatcher = CentralDispatcher(agent_id="dispatcher_1", message_bus=message_bus, state_subscriptions={'reservoir_level': RESERVOIR_STATE_TOPIC}, command_topics={'gate1_command': GATE_COMMAND_TOPIC}, rules=rules)

    return [reservoir, gate], [reservoir_twin, gate_twin, lca, dispatcher]

# Main simulation logic
simulation_dt = 1.0
simulation_config = {'duration': 1000, 'dt': simulation_dt}
harness = SimulationHarness(config=simulation_config)
message_bus = harness.message_bus

components, agents = setup_hierarchical_control_system(message_bus, simulation_dt)

# Create and add the disturbance agent
RAINFALL_TOPIC = "disturbance.rainfall.inflow"
rainfall_config = {"topic": RAINFALL_TOPIC, "start_time": 300, "duration": 200, "inflow_rate": 150}
rainfall_agent = RainfallAgent("rainfall_agent_1", message_bus, rainfall_config)

# Add all components and agents to the harness
for component in components:
    if hasattr(component, 'disturbance_topics'):
        component.disturbance_topics.append(RAINFALL_TOPIC)
    harness.add_component(component)

all_agents = agents + [rainfall_agent]
for agent in all_agents:
    harness.add_agent(agent)

harness.add_connection("reservoir_1", "gate_1")

# Build and run the simulation
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_mas_simulation()
sys.stdout = original_stdout

print("Disturbance simulation complete.")

## Results and Visualization

The plots show the system's state. A shaded region indicates the period of the rainfall disturbance. We can see the water level rise sharply during this period and observe the control system's response as it opens the gate wider to counteract the inflow.

In [ ]:
# Extract data from history
time = [h['time'] for h in harness.history]
reservoir_levels = [h['reservoir_1']['water_level'] for h in harness.history]
gate_openings = [h['gate_1']['opening'] for h in harness.history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Reservoir Level': reservoir_levels,
    'Gate Opening': gate_openings
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Reservoir Level
ax1.plot(df['Time'], df['Reservoir Level'], label='Reservoir Water Level')
ax1.axhline(y=12.0, color='red', linestyle='--', label='Flood Setpoint (12.0m)')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Hierarchical Control with Rainfall Disturbance')
ax1.grid(True)

# Add a shaded region for the rainfall event
ax1.axvspan(300, 500, color='blue', alpha=0.2, label='Rainfall Event')
ax1.legend()

# Gate Opening
ax2.plot(df['Time'], df['Gate Opening'], label='Gate 1 Opening', color='purple')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Gate Opening (m)')
ax2.grid(True)
ax2.axvspan(300, 500, color='blue', alpha=0.2, label='Rainfall Event')
ax2.legend()

plt.tight_layout()
plt.show()